# 79. Introduction to Performance Tuning of SQL

# 80. Overview of SQL Compilation Process and Explain Plans

# 81. Generate Explain Plans for SQL Queries

In [1]:
%%sql
EXPLAIN
SELECT * FROM orders WHERE order_id = 2;

,QUERY PLAN
0,Index Scan using orders_pkey on orders (cost=...
1,Index Cond: (order_id = 2)


In [5]:
%%sql
EXPLAIN
SELECT o.*,
    round(sum(oi.order_item_subtotal)::numeric, 2) AS revenue
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE o.order_id = 2
GROUP BY o.order_id,
         o.order_date,
         o.order_customer_id,
         o.order_status;

,QUERY PLAN
0,GroupAggregate (cost=0.29..3427.85 rows=1 wid...
1,-> Nested Loop (cost=0.29..3427.82 rows=4 ...
2,-> Index Scan using orders_pkey on or...
3,Index Cond: (order_id = 2)
4,-> Seq Scan on order_items oi (cost=...
5,Filter: (order_item_order_id = 2)


# 82. Review Tables used for Performance Tuning of SQL Queries

In [ ]:
%%sql
DROP INDEX orders_order_date_idx;
DROP INDEX order_items_oid_idx;

In [ ]:
%%sql
COMMIT;

In [ ]:
%%sql
ALTER TABLE order_items ADD
FOREIGN KEY (order_item_order_id) REFERENCES orders (order_id)

# 83. Review Data Storage Internals for Tables and Indexes

# 84. Review key terms used in Explain Plans for SQL Queries

# 85. Interpret Explain Plans for Basic SQL Queries

# 86. Review the Common Application Scenarios for Performance Tuning

# 87. Write SQL Queries for Customer Orders

In [8]:
%%sql
EXPLAIN
SELECT o.*,
       oi.*
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE order_customer_id = 5;

,QUERY PLAN
0,Hash Join (cost=1386.11..4827.13 rows=15 widt...
1,Hash Cond: (oi.order_item_order_id = o.order...
2,-> Seq Scan on order_items oi (cost=0.00.....
3,-> Hash (cost=1386.04..1386.04 rows=6 widt...
4,-> Seq Scan on orders o (cost=0.00.....
5,Filter: (order_customer_id = 5)


# 88. Performance Testing of SQL Queries using Stored Procedure

In [9]:
%%sql
SELECT count(*)
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE order_customer_id = 5;

,count
0,7


# 89. Add Required Indexes to tune performance of SQL Queries

In [ ]:
%%sql
ALTER TABLE orders
    ADD FOREIGN KEY (order_customer_id)
        REFERENCES customers (customer_id)

In [ ]:
%%sql
CREATE INDEX orders_order_customer_id_idx
ON orders (order_customer_id);

In [11]:
%%sql
EXPLAIN
SELECT count(*)
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE order_customer_id = 5;

,QUERY PLAN
0,Aggregate (cost=3467.62..3467.63 rows=1 width=8)
1,-> Hash Join (cost=26.56..3467.59 rows=15 ...
2,Hash Cond: (oi.order_item_order_id = o...
3,-> Seq Scan on order_items oi (cost=...
4,-> Hash (cost=26.49..26.49 rows=6 wi...
5,-> Bitmap Heap Scan on orders o...
6,Recheck Cond: (order_custo...
7,-> Bitmap Index Scan on o...
8,Index Cond: (order_c...


In [ ]:
%%sql
CREATE INDEX order_items_order_item_order_id_idx
ON order_items (order_item_order_id)

In [12]:
%%sql
EXPLAIN
SELECT count(*)
FROM orders AS o
    JOIN order_items AS oi
        ON o.order_id = oi.order_item_order_id
WHERE order_customer_id = 5;

,QUERY PLAN
0,Aggregate (cost=53.71..53.72 rows=1 width=8)
1,-> Nested Loop (cost=4.76..53.67 rows=15 w...
2,-> Bitmap Heap Scan on orders o (cos...
3,Recheck Cond: (order_customer_id...
4,-> Bitmap Index Scan on orders_...
5,Index Cond: (order_custome...
6,-> Index Only Scan using order_items_...
7,Index Cond: (order_item_order_id...


# 90. Guidelines on adding Indexes on Tables for SQL Queries